In [ ]:
# import libaries
import requests
import time
import pandas as pd
import re
import numpy as np
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

In [ ]:
# creating the path for the webdriver
PATH = "chromedriver.exe"
driver = webdriver.Chrome(executable_path=PATH)
chrome_options=Options()
chrome_options.add_argument("--headless")

# The wait functionality allows you to define a timeout to bound the length of the wait. Once this timeout is reached, the driver will stop its wait and move on to the next task
wait = WebDriverWait(driver, timeout=15)
wait
time.sleep(1)

# get the webpage of Wallapop
driver.get("https://es.wallapop.com/")
wait
time.sleep(1)

# accept the cookie preferences
try:
    cookies=WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))).click()
except:
    None
    
time.sleep(1)
    
# find the searchbar and click the category "Bicicletas" (first one appearing)
searchbar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchBoxForm"]/div/div[1]/input[1]')))
searchbar.send_keys("bicicleta")
firstCategory = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchBoxForm"]/div/div[3]/ul/li[1]/span[2]')))
firstCategory.click()
time.sleep(1)

In [ ]:
# find and click the location filter
filters=WebDriverWait(driver,timeout=10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='Bubble__content']")))
filters=driver.find_elements(By.XPATH, "//div[@class='Bubble__content']")
location=WebDriverWait(driver,timeout=10).until(EC.element_to_be_clickable(filters[2]))
location=filters[2]
location.click()

# select the location
time.sleep(2)
locationbutton=location.find_element(By.XPATH, "/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/div[1]/input")
locationbutton.click()
wait
locationbutton.clear()
wait
locationbutton.send_keys("Barcelona")

time.sleep(2)
try:
    locationresults=WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.TAG_NAME, "ngb-typeahead-window")))
    locationresults=driver.find_element(By.TAG_NAME, "ngb-typeahead-window")
except:
    locationresults=WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, "dropdown-menu show ng-star-inserted")))
    locationresults=driver.find_element(By.CLASS_NAME, "dropdown-menu show ng-star-inserted")
locationresults.find_element(By.XPATH,"//span[contains(text(), 'España, Barcelona')]").click()
time.sleep(2)

# move the slider to adjust the km 
slider = driver.find_element(By.CSS_SELECTOR, "body > tsl-root > tsl-public > div > div > tsl-search > div > div > tsl-filters-wrapper > div > div.FiltersWrapper__bar.d-flex.pl-3.py-2.FiltersWrapper__bar--opened > tsl-filter-group > div > div:nth-child(2) > tsl-filter-host > div > tsl-location-filter > tsl-filter-template > div > div > div.FilterTemplate__filter.px-4 > div > tsl-drawer-placeholder-template > div > div > form > tsl-slider-form > form > ngx-slider > span.ngx-slider-span.ngx-slider-pointer.ngx-slider-pointer-min")
move = ActionChains(driver)
# 100px = 1 movement; available steps: 1km, 5km, 10km, 30km, 50km, 100km, 200km, no limit
move.click_and_hold(slider).move_by_offset(-500,0).release().perform()
time.sleep(1)

# click apply button
location.find_element(By.XPATH, "//button[@class='btn btn-filter btn-primary']").click()

In [ ]:
# click price filter
price=WebDriverWait(driver,timeout=10).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), ' Precio ')]")))
price=driver.find_element(By.XPATH, "//div[contains(text(), ' Precio ')]")
price.click()

time.sleep(1)

# enter the max price 
maxprice=price.find_element(By.XPATH, "/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-range-filter/tsl-filter-template/div/div/div[2]/div/form/div/div[2]/input")
maxprice.click()
maxprice.clear()
maxprice.send_keys("800")
time.sleep(1)

#apply
price.find_element(By.XPATH, "/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-range-filter/tsl-filter-template/div/div/div[3]/tsl-button[2]/button").click()

In [ ]:
#subcategory filter
subcategory=WebDriverWait(driver,timeout=10).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), ' Subcategoría ')]")))
subcategory=driver.find_element(By.XPATH, "//div[contains(text(), ' Subcategoría ')]")
subcategory.click()
time.sleep(1)

# enter filters for bicileta types
biciletas_ciclos=driver.find_element(By.XPATH, "//p[contains(text(), ' Bicicletas y triciclos ')]")
biciletas_ciclos.click()
time.sleep(1)

#select the three types
element=driver.find_elements(By.XPATH, "//div[@class='MultiSelectOption d-flex align-items-center w-100 position-relative p-3']")
for item in element:
    if item.text.strip() in ["Bicicletas de carretera", "Bicicletas plegables", "MTB"]:
        item.find_element(By.TAG_NAME, "label").click()
    
# go back to subcategories
driver.find_element(By.XPATH, "/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[4]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-multi-select-form/div/div[1]/div").click()

# apply
driver.find_element(By.XPATH, "/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[4]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[3]/tsl-button[2]/button").click()

In [ ]:
# filter the condition
condition=WebDriverWait(driver,timeout=10).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), ' Estado del producto ')]")))
condition=driver.find_element(By.XPATH, "//div[contains(text(), ' Estado del producto ')]")
condition.click()
time.sleep(1)

#select the three conditions
element=driver.find_elements(By.XPATH, "//div[@class='MultiSelectOption d-flex align-items-center w-100 position-relative p-3']")
for item in element:
    if item.find_element(By.TAG_NAME, "p").text.strip() in ["Nuevo", "Como nuevo", "En buen estado"]:
        item.find_element(By.TAG_NAME, "label").click()
        
# click apply button 
driver.find_element(By.XPATH, "/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[3]/tsl-button[2]/button").click()

In [ ]:
ids = ["10438", "10217", "10214"]
condition = ["new", "as_good_as_new", "good"]

typesCount = 0
statesCount = 0

fixedUrl = "https://es.wallapop.com/app/search?category_ids=17000&keywords=bicicleta&filters_source=quick_filters&longitude=2.17001&latitude=41.38804&distance=10000&max_sale_price=800"

In [ ]:
time.sleep(2)

#create dataframe
df = pd.DataFrame(columns = ["Link", "Title", "Description", "Price", "Image", "Type", "State", "Children", "Size", "Size (letter)", "Date"])
    

#Loop through the 9 different combinations
for combination in range(9):
    
    #get page with combination bike-state
    driver.get(fixedUrl+"&object_type_ids="+ids[typesCount]+"&condition="+condition[statesCount])

    #set variables for scrolling
    scroll_pause_time = 1
    screen_height = driver.execute_script("return window.screen.height;")
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    items=driver.find_elements(By.XPATH, "//div[@class='ItemCard position-relative ItemCard--with-description ng-star-inserted']")
    i = 0

    #loop to scroll until when needed
    while True and len(items)<251: # set limit to stop when 250 items can be retrieved
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
        i += 1
        time.sleep(scroll_pause_time)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")
        if screen_height * i > scroll_height:
            try: 
                element=driver.find_element(By.XPATH, "//button[contains(text(),'Ver más productos')]")
                time.sleep(scroll_pause_time)
                driver.execute_script("arguments[0].click();", element)
                driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
            except:
                break
        WebDriverWait(driver,timeout=10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='ItemCard position-relative ItemCard--with-description ng-star-inserted']")))
        items=driver.find_elements(By.XPATH, "//div[@class='ItemCard position-relative ItemCard--with-description ng-star-inserted']")

    #retrieve all bikes
    grid=WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/tsl-search-layout/div/div[2]/div/tsl-public-item-card-list/div')))
    grid = driver.find_element(By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/tsl-search-layout/div/div[2]/div/tsl-public-item-card-list/div')
    bikes = grid.find_elements(By.TAG_NAME, "a")

    #define counter
    counter = 0

    #loop through all bikes and retrieve all information needed
    for bike in bikes:
        image = bike.find_element(By.TAG_NAME, "img")
        urlImage = image.get_attribute("src")

        bike.click()
        driver.switch_to.window(driver.window_handles[1])

        urlPost = driver.current_url
        try: 
            description = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[1]/p").text
        except:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            continue
    
        bikeType = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[1]/div[2]/div[2]/div[2]/span").text.strip()
        if bikeType=="Bicicletas de carretera":
            bikeType = "Bicicleta de carretera"
        elif bikeType=="Bicicletas plegables":
            bikeType = "Bicicleta plegable"

        state = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[1]/div[2]/div[1]/span").text.strip()
        if state=="Bueno":
            state = "En buen estado"

        date = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[2]/div[2]/div[1]").text.strip()

        if date.__contains__("ene"):
            date = date.replace("ene", "jan")
        elif date.__contains__("abr"):
            date = date.replace("abr", "apr")
        elif date.__contains__("ago"):
            date = date.replace("ago", "aug")
        elif date.__contains__("dic"):
            date = date.replace("dic", "dec")

        datetime = pd.to_datetime(date).date()

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

        title = bike.find_element(By.TAG_NAME, "p").text

        keywords = ["niño/a", "niño", "niña", "niños" , "niñas", "niño/as"]
        if any(word in title for word in keywords) or any(word in description for word in keywords):
            children = True
        else:
            children = False

        price = bike.find_element(By.TAG_NAME, "span").text.split('€')[0]
        try:
            price = float(price) #in the case of a non-decimal number
        except:
            price = float(price.replace(",", ".")) #in case of a decimal number that has ','

        if "talla " in description:
            size = description.split("talla ")[1].split(" ")[0]
            try:
                size = int(size)
                sizeLetter = np.nan
            except:
                sizeLetter = size.upper()
                size = np.nan
        elif "talla " in title:
            size = title.split("talla ")[1].split(" ")[0]
            try:
                size = int(size)
                sizeLetter = np.nan
            except:
                sizeLetter = size.upper()
                size = np.nan
        else:
            size = np.nan
            sizeLetter = np.nan
        if sizeLetter not in [np.nan, "XS", "S", "M", "L", "XL"]:
            sizeLetter = np.nan

        entry = pd.DataFrame([{"Link": urlPost,
                             "Title": title,
                             "Description": description,
                             "Price": price,
                             "Image": urlImage,
                              "Type": bikeType,
                              "State": state,
                              "Children": children,
                              "Size": size,
                              "Size (letter)": sizeLetter,
                              "Date": datetime}])
        df = pd.concat([df, entry], ignore_index = True)

        if counter == 249:
            break

        counter += 1
        
    if statesCount != 2:
        statesCount += 1
    else:
        typesCount += 1
        statesCount = 0
        
    
driver.close()

In [ ]:
df = df.drop_duplicates()
display(df)

In [ ]:
df.to_csv('group_A6_CSV_file.csv')

In [ ]:
#aggregate df and calculate mean
aggdf = df.groupby(["Type", "State"])["Price"].agg('mean')

types = []
states = []
avgPrice = []

#create new dataframe filled with aggregated values
for i in range(len(aggdf.index.values)):
    types.append(aggdf.index.values[i][0])
    states.append(aggdf.index.values[i][1])
    avgPrice.append(aggdf.values[i])
    
agg = pd.DataFrame({"Type": types,
                    "State": states,
                    "Avg Price": avgPrice})

#show final df
display(agg)